# Stock Prediction using RNNs

In [1]:
#Uncomment to install yfinance when running it the first time
!pip install yfinance

## Importing all necessary libraries

In [2]:
#Importing all libraries needed for the project.

#numpy is used for operations on arrays
import numpy as np

#pandas is used to perform operations on dataframes since yfinance return the data in dataframes. 
import pandas as pd

#yfinance is used to download the stock data from yahoo finance.
import yfinance as yf

#torch is used to create, train and test the neural networks
import torch

#sklearn is only used for the mean squared error function.
import sklearn

#matplotlib.pyplot is used for plotting the results.
import matplotlib.pyplot as plt

## Defining all functions needed

In [3]:
def download_dataset(stock='AAPL', start='1990-01-01', end='2022-01-01'):
    """This function downloads the stock data needed from yahoo finance in a certain time window.
    It then performs the necessary steps to calculate the day-to-day gain of the stock chosen from the stock's adjusted close.
    The stock gain column is then shifted up so as to create the values to predict for each day since we want to predict the next day's gain.
    The Vix and Nasdaq index are also added to the dataset. Preferably the stock would be listed on the Nasdaq exchange.
    Returns a pandas dataframe with the stock, vix, nsdq info.
    
    Arguments
    ---------
    stock : string
        The ticker of the stock we want to predict.
    start : string
        The start date of the dataset. Has to be the format YYYY-MM-DD (example:1990-01-01)
    end: string
        The end date of the dataset. Has to be the format YYYY-MM-DD (example:2022-01-01)
    
    Returns
    ---------
    full_dataset : pandas.core.frame.DataFrame
        Dataframe containing stock, vix and nasdaq price and volume information.
    
    Example
    --------
    >>> full_dataset = download_dataset('AMD', '1990-01-01', '2022-01-01')
    >>> full_dataset.shape
    >>> (8063, 18)
    
    """
    
    
    #Downloading stock data to be used
    stock_df=yf.download(stock, start=start, end=end, progress=True)
    
    #Downloading VIX index data to be used. VIX is a volatility index for the market.
    vix_df=yf.download('^VIX', start=start, end=end, progress=True)
    
    #Downloading NASDAQ index data to be used. NASDAQ is a stock exchange mainly for tech stocks.
    nsdq_df=yf.download('^IXIC', start=start, end=end, progress=True)
    
    #Renaming the Columns
    stock_df.rename(columns={'Open': 'Stock Open', 'High': 'Stock High', 'Low': 'Stock Low', 'Close': 'Stock Close', 'Adj Close': 'Stock Adj Close', 'Volume': 'Stock Volume'}, inplace=True)
    vix_df.rename(columns={'Open': 'Vix Open', 'High': 'Vix High', 'Low': 'Vix Low', 'Close': 'Vix Close', 'Adj Close': 'Vix Adj Close', 'Volume': 'Vix Volume'}, inplace=True)
    nsdq_df.rename(columns={'Open': 'Nasdaq Open', 'High': 'Nasdaq High', 'Low': 'Nasdaq Low', 'Close': 'Nasdaq Close', 'Nasdaq Close': 'Nasdaq Adj Close', 'Volume': 'Nasdaq Volume'}, inplace=True)
    
    #Adding column for the stock percentage change based on adjusted close dady-over-day. 
    stock_df['Stock Percentage Change'] = stock_df['Stock Adj Close'].pct_change()
    
    #Concatenating all dataset into one set.
    full_dataset=pd.concat([stock_df, vix_df, nsdq_df], axis=1)
    
    #Shifting up the stock percentage change. This will be used as the value to predict since we want to predict the change in the stock the following day.
    full_dataset['Stock Percentage Change'] = full_dataset['Stock Percentage Change'].shift(-1)
    
    #Dropping the column for the VIX index because the VIX does not have volume.
    full_dataset.drop(['Vix Volume'], axis=1, inplace=True)
    
    #Removing the last row because the Stock Percentage Change does not exist anymore since we shifted the column up
    full_dataset = full_dataset[:-1]
    
    return full_dataset

In [4]:
def create_dataset(full_dataset):
    """This function splits the dataset into X and y dataset consisting of the features and values to predict. 
    For X, this is done by dropping the Stock Percentage Change(value to predict) column from the dataset.
    For y, this is done by assigning y to be the Stock Percentage Change(value to predict) column from the dataset.
    
    Arguments
    ---------
    full_dataset : pandas.core.frame.DataFrame
        Dataframe containing stock, vix and nasdaq price and volume information.
    
    Returns
    ---------
    X: pandas.core.frame.DataFrame
        full_dataset dataframe with the Stock Percentage Change(value to predict) column dropped. The remaining columns are the features to train a machine learning model on.
    y : pandas.core.series.Series
        Stock Percentage Change(value to predict) column consisting of the values to predict of the dataset.
    
    Example
    --------
    >>> full_dataset.shape
    >>> (8063, 18)
    >>> X, y = create_dataset(full_dataset)
    >>> X.shape
    >>> (8063, 17)
    >>> y.shape
    >>> (8063,)
    
    """
    
    
    #Split data into X and y sets.
    X = full_dataset.drop(['Stock Percentage Change'], axis=1)
    y = full_dataset['Stock Percentage Change']
    
    return X, y

In [5]:
def mean_normalization(X, split_percentage=0.8):
    """This function normalizes all features of the dataset before the training/testing split. This is because the training/testing split is done on the timeseries created later.
    It is easier to normalize the data now rather than on the timeseries and the reults is the same. The mean and standard deviation is calculated on the training set only based on
    the split_percentage. 
    
    Arguments
    ---------
    X : pandas.core.frame.DataFrame
        DataFrame containing the dataset of features to train a machine learning model on.
    split_percentage : float
        Float representing the percentage of the dataset to use as the training set. 
        Used to determine the point up to where to calculate the mean and standard deviation used for the mean normalization.
    
    Returns
    ---------
    X_norm : pandas.core.frame.DataFrame
        Normalized dataset of all features(training and testing).
    X_norm_mean : pandas.core.series.Series
        the mean of all features of the training dataset.
    X_norm_std : pandas.core.series.Series
        the standard deviation of all features of the training dataset.
    
    Example
    --------
    >>> X.shape
    >>> (8063, 17)
    >>> X_norm, X_norm_mean, X_norm_std = mean_normalization(X, split_percentage)
    >>> X_norm.shape
    >>> (8063, 17)
    >>> X_norm_mean.shape
    >>> (17,)
    >>> X_norm_std.shape
    >>> (17,)
    
    """
    
    #Mean normalizing the training dataset based on the split percentage.
    norm_midpoint = int(X.shape[0]*split_percentage)
    X_norm_mean = X[:norm_midpoint].mean()
    X_norm_std = X[:norm_midpoint].std()
    X_norm=(X-X_norm_mean)/X_norm_std
    
    return X_norm, X_norm_mean, X_norm_std

In [6]:
def create_sequences(X_norm, y, window_shape=30):
    """This function creates timeseries of window_shape as datapoints to feed into a RNN using a rolling window of size window_shape. It also converts X and y into numpy arrays.
    
    Arguments
    ---------
    X_norm : pandas.core.frame.DataFrame
        Normalized dataset of all features(training and testing).
    y : pandas.core.series.Series
        Stock Percentage Change(value to predict) column consisting of the values to predict of the dataset.
    window_shape : int
        Integer corresponding to the length of the timeseries used for the prediction. Number corresponding to consecutive trading days.
    
    Returns
    ---------
    X_sequence : numpy.ndarray
        Numpy array corresponding to the sequence of window_shape days in the past of all features to train on.
    y_np : numpy.ndarray
        Numpy array corresponding to the values to predict of the dataset.
    
    Example
    --------
    >>> X_norm.shape
    >>> (8063, 17)
    >>> type(X_norm)
    >>> pandas.core.frame.DataFrame
    >>> y.shape
    >>> (8063,)
    >>> type(y)
    >>> pandas.core.series.Series
    >>> window_shape = 30
    >>> X_sequence, y_np = create_sequences(X_norm, y, window_shape)
    >>> X_sequence.shape
    >>> (8034, 30, 17)
    >>> type(X_sequence)
    >>> numpy.ndarray
    >>> y_np.shape
    >>> (8034,)
    >>> type(y_np)
    >>> numpy.ndarray
    
    """
    
    from numpy.lib.stride_tricks import sliding_window_view
    
    #Convert dataframes to numpy for easy computations
    X_np = X_norm.to_numpy()
    y_np = y.to_numpy()
    
    #Creating each sequence.
    X_sequence = sliding_window_view(X_np, window_shape = window_shape, axis=0)
    
    #Removing first window_shape-1 values to predict from the dataset because they are not associated to full sequences of features X.
    y_np = y_np[window_shape-1:]
    
    #Swapping the sequence length and the input size for the input into the RNN.
    X_sequence = np.swapaxes(X_sequence, 1, 2)
    
    return X_sequence, y_np

In [7]:
def split_dataset(X, y, split_percentage=0.8, validation_percentage=0.9):
    """This function splits the X and y into the training datasets X_train and y_train and the the testing datasets X_test and y_test. This is based on split_percentage which corresponds to the
    percentage of the dataset to be used as the training set. Further splits the training set into training and validation set based on the the validation_percentage.
    
    Arguments
    ---------
    X : pandas.core.frame.DataFrame
        Numpy array corresponding to the sequence of window_shape days in the past of all features to train on.
    y : numpy.ndarray
        Numpy array corresponding to the values to predict of the dataset.
    split_percentage : float
        Float representing the percentage of the dataset to use as the training set.
    validation_percentage : float
        Float representing the percentage of the training set to be used for training and the rest for validation.
    
    Returns
    ---------
    X_train : numpy.ndarray
        Numpy array corresponding to training features.
    y_train : numpy.ndarray
        Numpy array corresponding to training values to predict.
    X_val : numpy.ndarray
        Numpy array corresponding to validation features.
    y_val : numpy.ndarray
        Numpy array corresponding to the validation values to predict.
    X_test : numpy.ndarray
        Numpy array corresponding to testing features.
    y_test : numpy.ndarray
        Numpy array corresponding to testing values to predict.
    
    Example
    --------
    >>> X.shape
    >>> (8034, 17, 30)
    >>> y.shape
    >>> (8034,)
    >>> split_percentage = 0.8
    >>> validation_percentage = 0.9
    >>> X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(X, y, split_percentage, validation_percentage)
    >>> X_train.shape
    >>> (5784, 30, 17)
    >>> y_train.shape
    >>> (5784,)
    >>> X_val.shape
    >>> (643, 30, 17)
    >>> y_val.shape
    >>> (643,)
    >>> X_test.shape
    >>> (1607, 30, 17)
    >>> y_test.shape
    >>> (1607,)
    
    """
    
    #Splitting the dataset into training, validation and testing datasets.
    
    #Training and Testing split.
    split_point=int(X.shape[0]*split_percentage)
    X_train=X[:split_point]
    X_test=X[split_point:]
    y_train=y[:split_point]
    y_test=y[split_point:]
    
    #Training and Validation split.
    val_point = int(X_train.shape[0]*validation_percentage)
    X_val=X_train[val_point:]
    X_train=X_train[:val_point]
    y_val=y_train[val_point:]
    y_train=y_train[:val_point]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [8]:
def dataset_to_tensors(X_train, y_train, X_val, y_val, X_test, y_test): 
    """This function transforms all X_train, y_train, X_val, y_val, X_test, and y_test into tensors.
    
    Arguments
    ---------
    X_train : numpy.ndarray
        Numpy array corresponding to training features.
    y_train : numpy.ndarray
        Numpy array corresponding to training values to predict.
    X_val : numpy.ndarray
        Numpy array corresponding to validation features.
    y_val : numpy.ndarray
        Numpy array corresponding to the validation values to predict.
    X_test : numpy.ndarray
        Numpy array corresponding to testing features.
    y_test : numpy.ndarray
        Numpy array corresponding to testing values to predict.
    
    Returns
    ---------
    X_train_t : torch.Tensor
        Tensor corresponding to training features.
    X_test_t : torch.Tensor
        Tensor corresponding to testing features.
    X_val_t : torch.Tensor
        Tensor corresponding to validation features.
    y_val_t : torch.Tensor
        Tensor corresponding to the validation values to predict.
    y_train_t : torch.Tensor
        Tensor corresponding to training values to predict.
    y_test_t : torch.Tensor
        Tensor corresponding to testing values to predict.
    
    Example
    --------
    >>> type(X_train)
    >>> numpy.ndarray
    >>> type(X_val)
    >>> numpy.ndarray
    >>> type(X_test)
    >>> numpy.ndarray
    >>> type(y_train)
    >>> numpy.ndarray
    >>> type(y_val)
    >>> numpy.ndarray
    >>> type(y_test)
    >>> numpy.ndarray
    >>> X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t = dataset_to_tensors(X_train, y_train, X_val, y_val, X_test, y_test)
    >>> type(X_train_t)
    >>> torch.Tensor
    >>> type(X_test_t)
    >>> torch.Tensor
    >>> type(X_val_t)
    >>> torch.Tensor
    >>> type(y_train_t)
    >>> torch.Tensor
    >>> type(y_test_t)
    >>> torch.Tensor
    >>> type(y_val_t)
    >>> torch.Tensor
    
    """
    
    #Transforming the training dataset into tensors.
    X_train_t = torch.tensor(X_train).double()
    y_train_t = torch.tensor(y_train).double()
    y_train_t = y_train_t[:, None]
    
    #Transforming the validation dataset into tensors.
    X_val_t = torch.tensor(X_val).double()
    y_val_t = torch.tensor(y_val).double()
    y_val_t = y_val_t[:, None]
    
    #Transforming the testing dataset into tensors.
    X_test_t = torch.tensor(X_test).double()
    y_test_t = torch.tensor(y_test).double()
    y_test_t = y_test_t[:, None]
    
    return X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t

In [9]:
#RNN_network class that is used to initialize a RNN model.
class RNN_network(torch.nn.Module):
    def __init__(self, rnn_type = 'gru', input_size=17, hidden_size=5, num_layers=1, dropout=0):
        super(RNN_network, self).__init__()
        """The initialization method is used to initialize all the components needed to implement the neural model. We have to initialize a series
        of RNNs and a linear transformation. A choice of GRU or LSTM is given anything else defaults to a regular RNN.
        Template taken from Lab 5 Exercises for COMP 432 Machine Learning.
        
        """

        #Initialize the RNN.
        if rnn_type == 'gru':
            self.rnn = torch.nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type == 'lstm':
            self.rnn = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        else: 
            self.rnn = torch.nn.RNN(input_size, hidden_size, num_layers, batch_first=True)

        #Initialize the Linear transformation.
        self.linear = torch.nn.Linear(hidden_size, out_features=1) 
        
        #Introduce Dropout.
        self.dropout = torch.nn.Dropout(dropout)
        
    def forward(self, X):
        """This function processes the input sequence X with a series of RNNs. Then it applies
        a linear transformation on top of the last hidden state.

        Arguments
        ---------
        X : torch.Tensor
            Tensor containing the sequences (N, L, I).

        Returns
        ---------
        out : torch.Tensor
            Tensor containing the predictions.
        
        """

        #Run the RNN
        params, _ = self.rnn(X)
        Z = self.dropout(params)
        #Select the last hidden state
        z = Z[:,-1,:]
        #Apply linear transformation
        out = self.linear(z) 
        return out

In [10]:
def train_RNN(X_train_t, y_train_t, rnn_type = 'gru', lr = 0.0005, batch_size = 100, num_epoch = 500, hidden_size = 5, num_layers = 5, dropout = 0.25):
    """This function trains an RNN on the datasets X_train_t and y_train_t based on the passed hyperparameters. 
    Returns the a list of losses computed at for each batch and the trained RNN.
    Template taken from Lab 5 Exercises for COMP 432 Machine Learning.
    
    Arguments
    ---------
    rnn_type : string
        Choice of RNN ('gru', 'lstm' or anything else creates a default rnn).
    X_train_t : torch.Tensor
        Tensor corresponding to training features.
    y_train_t : torch.Tensor
        Tensor corresponding to training values to predict.
    lr : float
        Float corresponding to the learning rate.
    batch_size : int
        Integer corresponding to the batch size.
    num_epoch : int
        Integer corresponding to the number of epochs of the training.
    hidden_size : int
        Integer corresponding the hidden size of the RNN.
    num_layers : int
        Number of hidden layers in the RNN.
    dropout : float
        Amount of dropout as a percentage. Dropout refers to the number of neurons set to 0 at each layer. Reduces likelihood of overfitting, leads to better generalization.
    
    Returns
    ---------
    RNN : RNN_network
        Trained RNN network.
    losses : list
        List containing the losses computed for each batch.
    
    """
    
    
    #Hyperparameters
    input_size = X_train_t.shape[2]
    N = X_train_t.shape[0]

    #Storing the losses to plot them
    losses = []

    #Initialize the RNN 
    rnn = RNN_network(rnn_type = rnn_type, input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout).double()
    #Compute on GPU if available
    rnn = rnn.to(device)

    #Initialize the Loss.
    loss = torch.nn.MSELoss()

    #Initialize the Optimizer.
    optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

    #Training Loop
    for epoch in range(num_epoch):
        for i in range(0, N, batch_size):

            #Read minibatches (for both X and y)
            Xi = X_train_t[i:i+batch_size]
            yi = y_train_t[i:i+batch_size]

            #Run the model
            output = rnn(Xi)

            #Compute the loss 
            l = loss(output,yi) 

            #Update the parameters
            rnn.zero_grad()
            l.backward()
            optimizer.step()

            #Append losses to the losses list for further analysis.
            losses.append(l.item())

        #Print loss
        if (epoch+1) % 5 == 0:
            print("Epoch %03d: Train_loss: %.8f " %(epoch+1, l.item()))
            
    return rnn, losses

In [11]:
def predict(rnn, X_test_t):
    """This function uses an RNN to make prediction on a dataset X_test_t.
    
    Arguments
    ---------
    X_test_t : torch.Tensor
        Tensor corresponding to testing features.
    rnn : RNN_network
        Trained RNN network.
    
    Returns
    ---------
    y_pred : torch.Tensor
        The predictions made by the RNN.
    
    Example
    ---------
    >>> X_test_t.shape
    >>> torch.Size([1607, 30, 17])
    >>> y_pred = predict(rnn, X_test_t)
    >>> y_pred.shape
    >>> torch.Size([1607, 1])
    
    """
    
    #Put the model in eval mode
    rnn.eval()
    #Running the model on the testing set to get the predictions.
    y_pred=rnn(X_test_t)
    
    return y_pred

In [12]:
def plot_losses(losses, n):
    """This function plots the losses computed at each batch during the training of an RNN. It smooths the losses by calculation the moving average over n data points.
    This is done to eliminate outliers to better visualize the data.
    
    Arguments
    ---------
    losses : list
        List containing the losses computed for each batch during the training of an RNN.
    n : int
        Number of datapoints to use for the moving average.
    
    """
    
    #Takes the losses and computes the moving average over n values.
    convolved = np.convolve(losses, np.ones(n)/n, mode='valid')
    #Plots the losses
    plt.figure(figsize=(15, 10))
    plt.title('Losses for each batch computed')
    plt.ylabel('MSE loss')
    plt.xlabel('')
    plt.plot(convolved, label="losses")
    plt.legend

In [13]:
def separate_plot_predicted_vs_actual_gains(y_test_t, y_pred):
    """This function plots two lines, on different figures, which correspond to the Predicted day-to-day % gain and the Actual day-to-day % gain.
    
    Arguments
    ---------
    y_test_t : torch.Tensor
        Tensor corresponding to testing values to predict.
    y_pred : torch.Tensor
        The predictions made by the RNN.
    
    """
    
    #Plotting the Predicted day-to-day %gain.
    plt.figure(figsize=(15,5))
    #Set same limits as the plot for the actual gains
    plt.ylim([y_test_t.cpu().detach().min(), y_test_t.cpu().detach().max()])
    plt.plot(y_pred.cpu().detach(), label="Predicted Gain")
    plt.title('Predicted Gain Over Time')
    plt.xlabel("time steps")
    plt.ylabel('predicted % gain')
    plt.legend()
    
    #Plotting the Actual day-to-day %gain.
    plt.figure(figsize=(15,5))
    plt.plot(y_test_t.cpu().detach(), label="Actual Gain")
    plt.title('Actual Gain Over Time')
    plt.xlabel("time steps")
    plt.ylabel('actual % gain')
    plt.legend()

In [14]:
def same_plot_predicted_vs_actual_gains(y_test_t, y_pred):
    """This function plots two lines, on the same figure, which correspond to the Predicted day-to-day % gain and the Actual day-to-day % gain.
    
    Arguments
    ---------
    y_test_t : torch.Tensor
        Tensor corresponding to testing values to predict.
    y_pred : torch.Tensor
        The predictions made by the RNN.
    
    """
    
    #Plotting the Actual day-to-day %gain.
    plt.figure(figsize=(30,10))
    plt.plot(y_test_t.cpu().detach(), label="Actual Gain", color='orange')
    plt.title('Gain Over Time')
    plt.xlabel("time steps")
    plt.ylabel('% gain')
    plt.legend()
    
    #Plotting the Predicted day-to-day %gain.
    plt.plot(y_pred.cpu().detach(), label="Predicted Gain")
    plt.legend()

In [15]:
def mse(y_test_t, y_pred): 
    """This function computes the MSE of the Predicted day-to-day %gain versus the Actual day-to-day %gain.
    
    Arguments
    ---------
    y_test_t : torch.Tensor
        Tensor corresponding to testing values to predict.
    y_pred : torch.Tensor
        The predictions made by the RNN.
    
    Returns
    ---------
    mse : numpy.float64
        Mean squared error of the computed predictions vs the actual % gain.
    
    Example
    ---------
    >>> y_test_t.shape
    >>> torch.Size([1607, 1])
    >>> y_pred = predict(rnn, X_test_t)
    >>> y_pred.shape
    >>> torch.Size([1607, 1])
    
    """
    
    #Mean squared error of predictions.
    from sklearn.metrics import mean_squared_error
    
    mse = mean_squared_error(y_test_t.cpu().detach(), y_pred.cpu().detach())
    
    return mse

In [16]:
def hyperparameter_3d_search(X_train_t, y_train_t, X_val_t, y_val_t, rnn_type, lr, hidden_size, num_layers):
    """This function performs a hyperparameter search for an RNN on the learning rate, the hidden_size and the number of layers.
    
    Arguments
    ---------
    X_train_t : torch.Tensor
        Tensor corresponding to training features.
    y_train_t : torch.Tensor
        Tensor corresponding to training values to predict.
    X_val_t : torch.Tensor
        Tensor corresponding to validation features.
    y_val_t : torch.Tensor
        Tensor corresponding to the validation values to predict.
    rnn_type : string
        Choice of RNN ('gru', 'lstm' or anything else creates a default rnn).
    lr : list
        List of floats corresponding to the learning rates.
    hidden_size : list
        List of integers corresponding the hidden sizes of the RNN.
    num_layers : list
        List with the number of hidden layers in the RNN.
    
    Returns
    ---------
    best_rnn : RNN_network
        Trained RNN network with the lowest mse on test on the validation set.
    best_mse : float
        Lowest mse calculated.
    best_losses : list
        losses calculated at each batch of the best RNN.
    params : list
        List of dictionaries containing all the hyperparamets tested for each rnn along with their mse. Sorted from lowest to highest mse.
    
    """
    
    #Variables used.
    current_mse = 0
    params = []
    best_mse = 1000
    best_rnn = None
    best_losses = []
    
    #Hyperparameter search loop.
    for rate in lr:
        for size in hidden_size:
            for num in num_layers:
                #Train the RNN.
                rnn, losses = train_RNN(X_train_t, y_train_t, rnn_type = rnn_type, lr = rate, batch_size = 100, num_epoch = 1000, hidden_size = size, num_layers = num, dropout = 0.25)
                
                #Performing the prediction on the validation set.
                y_pred = predict(rnn, X_val_t)
                
                #Calculating the mean squared error of the predicted vs actual % gain on the validation set.
                current_mse = mse(y_val_t, y_pred)
                
                #Storing dictionary of parameters and mse.
                params.append({'mse': current_mse, 'lr': rate, 'hidden_size': size, 'num_layers': num})
                
                #Writing the parameters to a file each training loop.
                with open('params.txt', 'w') as f:
                    f.write(',\n'.join(str(i) for i in params))
                
                #Testing for the best RNN.
                if current_mse < best_mse:
                    best_mse = current_mse
                    best_rnn = rnn
                    best_losses = losses
                    
    #Sorting the parameters list from best to worst based on mse.               
    params = sorted(params, key=lambda i: i['mse'])
    
    #Writing the sorted parameters to a file at the end of hyperparameter search.
    with open('sorted_params.txt', 'w') as f:
        f.write(',\n'.join(str(i) for i in params))
                
    return best_rnn, best_mse, best_losses, params

In [17]:
def simulation(y_test_t, y_pred, threshold=0.0):
    """This function simulates trading on based on the predictions with $200,000 and plots a graph representing each trade.
    Trades are made when predicting a gain above a certain threshold.
    The trades are plotted and compared to just holding the stock over the same time period.
    
    Arguments
    ---------
    y_test_t : torch.Tensor
        Tensor corresponding to testing values to predict.
    y_pred : torch.Tensor
        The predictions made by the RNN.
    threshold : float
        Float corresponding to the percent gain used as a threshold. Trades are made when a value above this threshold is predicted.
    
    """
    
    #Transforming tensors to numpy arrays.
    y_test, y_pred = y_test_t.cpu().detach().numpy().flatten(), y_pred.cpu().detach().numpy().flatten()
    
    #Variables used.
    initial = 200000
    hold_profits = y_test
    pred_profits = y_pred
    
    #Loop where we perform the trades.
    for i in range(hold_profits.size):
        if i == 0:
            if pred_profits[i] > threshold:
                pred_profits[i] = initial+y_test[i]*initial
            else: 
                pred_profits[i] = initial
            hold_profits[i] = initial+y_test[i]*initial
        
        else: 
            if pred_profits[i] > threshold:
                pred_profits[i] = pred_profits[i-1] + pred_profits[i-1]*y_test[i]
            else: 
                pred_profits[i] = pred_profits[i-1]
            hold_profits[i] = hold_profits[i-1] + hold_profits[i-1]*y_test[i]
    
    #Plotting the Actual day-to-day %gain.
    plt.figure(figsize=(30,10))
    plt.plot(pred_profits, label="Total dollar amount using predictions ", color='orange')
    plt.title('Total Dollar Amount over Time (starting with $200,000)')
    plt.xlabel("time steps")
    plt.ylabel('total $')
    plt.ticklabel_format(style='plain')
    plt.legend(prop={'size': 10})
    
    #Plotting the Predicted day-to-day %gain.
    plt.plot(hold_profits, label="Total dollar amount from holding")
    plt.legend()
    
    #Printing final profits from holding versus trading based on the predictions.
    print("Total dollar amount from holding: ", hold_profits[-1])
    print("Total dollarr amount from trading using prediction: ", pred_profits[-1])

In [18]:
def plot_hyperparameter_search_results(params):
    """This function displays one 3d scatter plot where the x,y,z axes correspond 
    to the learning rate, hidden_size, and number of layers. 
    The darker points are the ones with the lower mse.
    
    Arguments
    ---------
    params : list
        A list of dictionaries containing mse, learning rates, hidden sizes and number of layers.
    
    """

    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    values = [] 
    for i in params:
        values.append(i['mse'])
    # Mean mse will be used to remove outliers from the graph so we get a better picture of possible patterns.
    mean_mse = np.mean(values)

    mse_values = []
    for i in params:
        if i['mse'] < mean_mse:
            mse_values.append(i['mse'])

    lr_data = []
    for i in params:
        if i['mse'] < mean_mse:
            lr_data.append(i['lr'])

    hs_data = []
    for i in params:
        if i['mse'] < mean_mse:
            hs_data.append(i['hidden_size'])

    nl_data = []
    for i in params:
        if i['mse'] < mean_mse:
            nl_data.append(i['num_layers'])

    values = [] 
    for i in params:
        if i['mse'] < mean_mse:
            values.append(i['mse'])

    #taking the log of the values so that differences are more apparent.
    mse_values = np.log(mse_values)

    img = ax.scatter(lr_data, hs_data, nl_data, c=mse_values, s=500, cmap=plt.hot(), alpha=1)
    plt.xlabel('learning rate')
    plt.ylabel('hidden size')
    ax.set_zlabel('number of layers')
    plt.title('MSE of RNN on Validation Set with Different Hyperparameters')
    fig.colorbar(img)

    plt.show()

In [19]:
def plot_learning_rate_hyperparameter_search_results(params):
    """This function displays one 3d scatter plot for each learning rate. 
    The x,y,z axes correspond to the hidden_size, number of layers and the mse.
    The darker points are the ones with the lower mse.
    
    Arguments
    ---------
    params : list
        A list of dictionaries containing mse, learning rates, hidden sizes and number of layers.
    
    """
    
    values = [] 
    for i in params:
        values.append(i['mse'])
    
    # Mean mse will be used to remove outliers from the graph so we get a better picture of possible patterns.
    mean_mse = np.mean(values)

    mse_values = []
    for i in params:
        if i['mse'] < mean_mse:
            mse_values.append(i['mse'])

    lr_data = []
    for i in params:
        if i['mse'] < mean_mse:
            lr_data.append(i['lr'])

    hs_data = []
    for i in params:
        if i['mse'] < mean_mse:
            hs_data.append(i['hidden_size'])

    nl_data = []
    for i in params:
        if i['mse'] < mean_mse:
            nl_data.append(i['num_layers'])

    mse_data = [] 
    for i in params:
        if i['mse'] < mean_mse:
            mse_data.append(i['mse'])

    lr_set = set(lr_data)
    for l in lr_set:
        mse_values = []
        hs_values = []
        nl_values = []
        for i, lr in zip(range(len(lr_data)),lr_data):
            if lr == l:
                hs_values.append(hs_data[i])
                nl_values.append(nl_data[i])
                mse_values.append(mse_data[i])
        hs_values = np.array(hs_values)
        nl_values = np.array(nl_values)
        mse_values = np.array(mse_values)
        mse_values = mse_values[:, None]
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        img = ax.scatter(hs_values, nl_values, mse_values, c=mse_values, s=2000, cmap=plt.hot(), alpha=1)
        plt.xlabel('hidden size')
        plt.ylabel('number of layers')
        ax.set_zlabel('mse')
        plt.title("MSE of RNN with a Learning Rate of %.4f on Validation Set" %(l))
        fig.colorbar(img)

## Main Code

### Variables and seed initialization

In [20]:
#Using GPU for computation if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [21]:
#Set all seeds to 0 for reproducibility
torch.manual_seed(0)
np.random.seed(0)

In [22]:
#Initializing all needed variables

#Stock Ticker
stock = 'AAPL'

#Start and end date of dataset
start = '1990-01-01'
end = '2022-01-01'

#Percentage used to split the training and testing data. 
split_percentage = 0.8

#Percentage used to split the training and validation data.
validation_percentage = 0.9

#Window shape is the number of days that we want each time slice to be (sequence length).
window_shape = 30

### Data importing and preprocessing

In [23]:
#Downloading the data and creating the dataset.
full_dataset = download_dataset(stock, start, end)

In [24]:
#Split data into X and y sets.
X, y = create_dataset(full_dataset)

In [25]:
#Normalizing the data.
X_norm, X_norm_mean, X_norm_std = mean_normalization(X, split_percentage)

In [26]:
#Creating the sequences of window_shape days.
X, y = create_sequences(X_norm, y, window_shape)

In [27]:
#Splitting dataset into training and testing sets.
X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(X, y, split_percentage, validation_percentage)

In [28]:
#Converting training and testing sets to tensors.
X_train_t, y_train_t, X_val_t, y_val_t, X_test_t, y_test_t = dataset_to_tensors(X_train, y_train, X_val, y_val, X_test, y_test)

In [29]:
#Making sure that the dataset is sent to the GPU if available.
X_train_t=X_train_t.to(device)
y_train_t=y_train_t.to(device)
X_val_t=X_val_t.to(device)
y_val_t=y_val_t.to(device)
X_test_t=X_test_t.to(device)
y_test_t=y_test_t.to(device)

### RNN training

In [30]:
#Training the RNN without hyperparater search.
rnn, losses = train_RNN(X_train_t, y_train_t, rnn_type = 'gru', lr = 0.0005, batch_size = 100, num_epoch = 1000, hidden_size = 10, num_layers = 2, dropout = 0.25)

In [31]:
#Hyperparameters list
rnn_type = 'gru'
lr = [0.0005, 0.005, 0.05]
hidden_size = [10, 20, 30]
num_layers = [1, 2, 3]

In [32]:
#Hyperparameter training.
# rnn, mse_result, losses, params = hyperparameter_3d_search(X_train_t, y_train_t, X_val_t, y_val_t, rnn_type = rnn_type, lr = lr, hidden_size = hidden_size, num_layers = num_layers)

In [33]:
#Plotting the losses.
plot_losses(losses, 300)

### RNN testing

In [34]:
#Performing the predictions on the test set.
y_pred = predict(rnn, X_test_t)

In [35]:
#Plotting the training predictions vs the training set percentage gain.
separate_plot_predicted_vs_actual_gains(y_train_t, predict(rnn, X_train_t))

In [36]:
#Plotting the predicted vs the test set percentage gain.
same_plot_predicted_vs_actual_gains(y_test_t, y_pred)

In [37]:
#Calculating the mean squared error of the predicted vs actual % gain on the test set.
mse(y_test_t, y_pred)

In [38]:
#Running a simulation on $200,000 to see how the RNN performs on the test set.
simulation(y_test_t, y_pred, threshold=0.0)

In [42]:
#Running a simulation on $200,000 to see how the RNN performs on the validation set.
simulation(y_val_t, predict(rnn, X_val_t), threshold=0.0)

### Hyperparameter Plotting

In [39]:
#Plotting the results of the hyperparamter search.
# plot_hyperparameter_search_results(params)

In [40]:
#Plotting the results of the hyperparameter search for each learning rate.
# plot_learning_rate_hyperparameter_search_results(params)